In [68]:
import sys

In [69]:
def bellmanFord(graph, src, dest):
    inf = sys.maxsize
    node_data = {
        'USD':{'cost': inf, 'pred': []},
        'YEN':{'cost': inf, 'pred': []},
        'POUND':{'cost': inf, 'pred': []},
    }
    node_data[src]['cost'] = 0
    for i in range(2):
        print(f'Iteration {i}:')
        
        for itr in graph:
            
            for neighbor in graph[itr]:
                cost = node_data[itr]['cost'] + graph[itr][neighbor]
                
                if cost < node_data[neighbor]['cost']:
                    node_data[neighbor]['cost'] = cost
                    
                    if node_data[neighbor]['cost'] == inf:
                        node_data[neighbor]['pred'].append(''.join(list(itr)))
                        
                    else:
                        node_data[neighbor]['pred'].clear()
                        node_data[neighbor]['pred'].append(''.join(list(itr)))
                        
        print(node_data)
    
    node_data[dest]['pred'].append(''.join(list(dest)))
    print('Shortest Distance: ' + str(node_data[dest]['cost']))
    print('Shortest Path: ' + str(node_data[dest]['pred']))

In [70]:
graph = {
    'USD':{'POUND': -log(0.80)},
    'POUND':{'YEN': -log(100)},
    'YEN':{'USD': -log(0.013)}
}

In [71]:
source = 'USD'
destination = 'YEN'

In [72]:
bellmanFord(graph, source, destination)

Iteration 0:
{'USD': {'cost': -0.03922071315328157, 'pred': ['YEN']}, 'YEN': {'cost': -4.382026634673882, 'pred': ['POUND']}, 'POUND': {'cost': 0.2231435513142097, 'pred': ['USD']}}
Iteration 1:
{'USD': {'cost': -0.07844142630656314, 'pred': ['YEN']}, 'YEN': {'cost': -4.421247347827164, 'pred': ['POUND']}, 'POUND': {'cost': 0.18392283816092814, 'pred': ['USD']}}
Shortest Distance: -4.421247347827164
Shortest Path: ['POUND', 'YEN']


In [73]:
from typing import Tuple, List
from math import log


# rates = [base curremcy = 1, x currency in base currency, ...], [ x currency in base currency, base curremcy = 1,  x currency in base currency, ...]
rates = [
    [1, 0.23, 0.25, 16.43, 18.21, 4.94], # [PLN->PLN, PLN->EUR, PLN->USD, ...]
    [4.34, 1, 1.11, 71.40, 79.09, 21.44],# [EUR->PLN, EUR->EUR, EUR->USD, ...]
    [3.93, 0.90, 1, 64.52, 71.48, 19.37],
    [0.061, 0.014, 0.015, 1, 1.11, 0.30],
    [0.055, 0.013, 0.014, 0.90, 1, 0.27],
    [0.20, 0.047, 0.052, 3.33, 3.69, 1],
]

currencies = (
    'PLN', 
    'EUR', 
    'USD', 
    'RUB', 
    'INR', 
    'MXN'
)


def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) for edge in row] for row in graph]
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                print("Arbitrage Opportunity: \n")
                print(" --> ".join([currencies[p] for p in print_cycle[::-1]]))
                
def returnOpp(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    opportunities = []
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                opportunities.append([currencies[p] for p in print_cycle[::-1]])
    return opportunities

def calculateProfit():
    oppurtunities = returnOpp(currencies, rates)
    for i in oppurtunities:
        print(i)
    


if __name__ == "__main__":
    arbitrage(currencies, rates)
    calculateProfit()

Arbitrage Opportunity: 

RUB --> INR --> PLN --> RUB
Arbitrage Opportunity: 

USD --> MXN --> USD --> RUB --> INR --> EUR --> PLN
Arbitrage Opportunity: 

USD --> MXN --> USD --> PLN
['RUB', 'INR', 'PLN', 'RUB']
['USD', 'MXN', 'USD', 'RUB', 'INR', 'EUR', 'PLN']
['USD', 'MXN', 'USD', 'PLN']


In [24]:
from typing import Tuple, List
from math import log


# rates = [base curremcy = 1, x currency in base currency, ...], [ x currency in base currency, base curremcy = 1,  x currency in base currency, ...]
rates = [
    [1, 0.000048101089, 0.000882682649, 2.096436058700, 0.026546323334, 1.819505094614],
    [20789.55, 1, 18.350575067746, 43583.962264150900, 551.886116272896, 37826.692139738000],
    [1132.91, 0.054494205021, 1, 2375.073375262060, 30.074595168569, 2061.335516739450],
    [0.477, 0.000022944220, 0.000421039624, 1, 0.012662596230, 0.867903930131],
    [37.67, 0.001811968032, 0.033250655392, 78.972746331237, 1, 68.540756914119],
    [0.5496, 0.000026436359, 0.000485122384, 1.152201257862, 0.014589859304, 1],
]

currencies = (
    'USDT', 
    'BTC', 
    'ETH', 
    'ADA', 
    'SOL', 
    'MATIC'
)


def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) for edge in row] for row in graph]
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                print("Arbitrage Opportunity: \n")
                print(" --> ".join([currencies[p] for p in print_cycle[::-1]]))


if __name__ == "__main__":
    arbitrage(currencies, rates)

Arbitrage Opportunity: 

ADA --> ETH --> MATIC --> ADA --> BTC --> USDT
Arbitrage Opportunity: 

ADA --> ETH --> ADA
Arbitrage Opportunity: 

ETH --> MATIC --> ADA --> ETH --> SOL
Arbitrage Opportunity: 

ETH --> MATIC --> ADA --> ETH --> USDT
Arbitrage Opportunity: 

ADA --> ETH --> MATIC --> ADA --> USDT
Arbitrage Opportunity: 

ADA --> ETH --> ADA
Arbitrage Opportunity: 

ETH --> MATIC --> ADA --> ETH --> SOL


# Let's suppose we start with 10000 USD then exchange it for MXN. We get 49400 MXN.
## Amount of USD * EUR exchange rate = 49400

# Now I want to exchange this with USD.
## Amount of MXN * USD exchange rate = 

# 7410 EUR = 7410 * 1.366 = 10122 CAD
# Now let's exchange this for USD.
# 10122 CAD = 10122 * 0.995 = 10071
# We started with 10000 USD and ended up having 10071 USD. That is a good 71 USD profit. 

In [67]:
rates = [
    {"PLN": 1, "EUR": 0.23, "USD": 0.25, "RUB": 16.43, "INR": 18.21, "MXN": 4.94}, # [PLN->PLN, PLN->EUR, PLN->USD, ...]
    {"PLN": 4.34, "EUR": 1, "USD": 1.11, "RUB": 71.40, "INR": 79.09, "MXN": 21.44},# [EUR->PLN, EUR->EUR, EUR->USD, ...]
    {"PLN": 3.93, "EUR": 0.90, "USD": 1, "RUB": 64.52, "INR": 71.48, "MXN": 19.37},
    {"PLN": 0.061, "EUR": 0.014, "USD": 0.015, "RUB": 1, "INR": 1.11, "MXN": 0.30},
    {"PLN": 0.055, "EUR": 0.013, "USD": 0.014, "RUB": 0.90, "INR": 1, "MXN": 0.27},
    {"PLN": 0.20, "EUR": 0.047, "USD": 0.052, "RUB": 3.33, "INR": 3.69, "MXN": 1},
]